In [260]:
# pip install pandas
# pip install plotly
# pip install os
# pip install datetime
# pip install -U kaleido

In [261]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from datetime import datetime
from scipy.stats import zscore

In [262]:
df = {}

for archive in os.listdir('dados/dados_fornec'):
    data = pd.read_csv(f'dados/dados_fornec/{archive}')
    df[f'{archive.split(".")[0]}'] = pd.DataFrame(data)

## Explorando os Datasets

### Agências

In [263]:
df['agencias'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cod_agencia    10 non-null     int64 
 1   nome           10 non-null     object
 2   endereco       10 non-null     object
 3   cidade         10 non-null     object
 4   uf             10 non-null     object
 5   data_abertura  10 non-null     object
 6   tipo_agencia   10 non-null     object
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes


Perguntas:
- Quantas agências foram abertas por ano?

### Clientes

In [264]:
df['clientes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_cliente      998 non-null    int64 
 1   primeiro_nome    998 non-null    object
 2   ultimo_nome      998 non-null    object
 3   email            998 non-null    object
 4   tipo_cliente     998 non-null    object
 5   data_inclusao    998 non-null    object
 6   cpfcnpj          998 non-null    object
 7   data_nascimento  998 non-null    object
 8   endereco         998 non-null    object
 9   cep              998 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.1+ KB


Perguntas:
- Qual a faixa etária dos clientes?

### Colaborador-Agência

In [265]:
df['colaborador_agencia'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   cod_colaborador  100 non-null    int64
 1   cod_agencia      100 non-null    int64
dtypes: int64(2)
memory usage: 1.7 KB


Perguntas:
- Quantos coleboradores trabalham em cada agência?

### Colaboradores

In [266]:
df['colaboradores'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cod_colaborador  100 non-null    int64 
 1   primeiro_nome    100 non-null    object
 2   ultimo_nome      100 non-null    object
 3   email            100 non-null    object
 4   cpf              100 non-null    object
 5   data_nascimento  100 non-null    object
 6   endereco         100 non-null    object
 7   cep              100 non-null    object
dtypes: int64(1), object(7)
memory usage: 6.4+ KB


Perguntas:
- Qual a faixa etária dos colaboradores?
- Qual a distribuição dos colaboradores por gênero?

### Contas

In [267]:
df['contas'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_conta               999 non-null    int64  
 1   cod_cliente             999 non-null    int64  
 2   cod_agencia             999 non-null    int64  
 3   cod_colaborador         999 non-null    int64  
 4   tipo_conta              999 non-null    object 
 5   data_abertura           999 non-null    object 
 6   saldo_total             999 non-null    float64
 7   saldo_disponivel        999 non-null    float64
 8   data_ultimo_lancamento  999 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 70.4+ KB


Perguntas:
- Quantas contas cada agência possui?
- Quantas contas cada colaborador abriu/gerencia?
- Quantas contas foram abertas por ano?
- Quantas contas ainda estão ativas?
- Qual o perfil dos clientes?
- Quais colaboradores gerenciam mais contas?
- Quais colaboradores gerenciam menos contas?

#### Quantidade de contas por agência

In [268]:
contas_por_agencia = df['contas']['cod_agencia'].value_counts() # estudar quais agências tem maior movimentação

#### Quantidade de contas por colaborador

In [269]:
contas_por_colaborador = df['contas']['cod_colaborador'].value_counts() # estudar a eficiência de cada colaborador e relacionar a sua eficiência à movimentação da agência

#### Distribuição das Contas por agência

In [303]:
nomes_agencias = {df['agencias']['cod_agencia'][i]: df['agencias']['nome'][i] for i in df['agencias'].index}

contas_por_agencia_df = pd.DataFrame(contas_por_agencia)

contas_por_agencia_df = contas_por_agencia_df.reset_index()

contas_por_agencia_df = contas_por_agencia_df.rename(columns={'index': 'agencia', 'cod_agencia': 'contas'})

contas_por_agencia_df['nome_agencia'] = contas_por_agencia_df['agencia'].map(nomes_agencias)

fig = px.pie(contas_por_agencia_df, values='contas', names='nome_agencia', title='Distribuição das Contas por Agência')

fig.update_layout(width=1200, height=600)

fig.show()

fig.write_image('plots_export/contas_agencia_pie.png')
fig.write_html('plots_export_html/contas_agencia_pie.html')

#### Classificação dos clientes por saldo disponível

In [271]:
q1 = np.percentile(df['contas']['saldo_disponivel'], 25)  # primeiro quartil (25%)
q2 = np.percentile(df['contas']['saldo_disponivel'], 50)  # segundo quartil (mediana, 50%)
q3 = np.percentile(df['contas']['saldo_disponivel'], 75)  # terceiro quartil (75%)

iqr = q3 - q1

lower_bound = q1 - (1.5 * iqr) if q1 - (1.5 * iqr) > 0 else 0
upper_bound = q3 + (1.5 * iqr)

print(f"Lower Bound: {lower_bound},\nQ1: {q1},\nMediana (Q2): {q2},\nQ3: {q3},\nUpper Bound: {upper_bound}")

Lower Bound: 0,
Q1: 918.4850000000007,
Mediana (Q2): 11325.959999999995,
Q3: 33926.209999999985,
Upper Bound: 83437.79749999996


In [272]:
labels = ['baixa', 'média-baixa', 'média', 'média-alta', 'alta']

bins = [lower_bound, q1, q2, q3, upper_bound, df['contas']['saldo_disponivel'].max()]

df['contas']['classificacao'] = pd.cut(df['contas']['saldo_disponivel'], bins=bins, labels=labels, include_lowest=True)

In [273]:
df['contas']['classificacao'].value_counts()

baixa          250
média-baixa    250
média          249
média-alta     180
alta            70
Name: classificacao, dtype: int64

Visualização dos dados pós-classificação

In [274]:
fig = px.scatter(df['contas'], x=df['contas'].index, y='saldo_disponivel', 
                 color='classificacao')

fig.show()

#### Evolução da abertura de contas por ano

In [275]:
ano_abertura = []
ano_ultimo_lanc = []

for i in df['contas'].index:
    ano_abertura.append(int(datetime.strptime(df['contas']['data_abertura'][i].split()[0], "%Y-%m-%d").strftime("%Y")))
    ano_ultimo_lanc.append(int(datetime.strptime(df['contas']['data_ultimo_lancamento'][i].split()[0], "%Y-%m-%d").strftime("%Y")))

df['contas']['ano_abertura'] = ano_abertura
df['contas']['ano_ultimo_lanc'] = ano_ultimo_lanc

ano_abert_df = pd.DataFrame(df['contas']['ano_abertura'].value_counts())
ano_abert_df = ano_abert_df.reset_index()
ano_abert_df = ano_abert_df.rename(columns={'index': 'ano_abert', 'ano_abertura':'valores'})
ano_abert_df = ano_abert_df.sort_values(by='ano_abert')

fig = px.line(ano_abert_df, x='ano_abert', y='valores',
              labels={'valores': 'Volume', 'ano_abert': 'Ano de Abertura'},
              title='Evolução Anual de Abertura de Contas')

fig.show()

#### Atividade de Clientes

In [276]:
tempo_inativo = []
for i in df['contas'].index:
    t = datetime.strptime(df['contas']['data_ultimo_lancamento'][i].split()[0], "%Y-%m-%d") - datetime.strptime(df['contas']['data_abertura'][i].split()[0], "%Y-%m-%d")
    tempo_inativo.append(t.days)

df['contas']['t_inat'] = tempo_inativo

In [277]:
q1 = np.percentile(df['contas']['t_inat'], 25)  # primeiro quartil (25%)
q2 = np.percentile(df['contas']['t_inat'], 50)  # segundo quartil (mediana, 50%)
q3 = np.percentile(df['contas']['t_inat'], 75)  # terceiro quartil (75%)

iqr = q3 - q1

lower_bound = q1 - (1.5 * iqr) if q1 - (1.5 * iqr) > 0 else 0
upper_bound = q3 + (1.5 * iqr)

print(f"Lower Bound: {lower_bound},\nQ1: {q1},\nMediana (Q2): {q2},\nQ3: {q3},\nUpper Bound: {upper_bound}")

Lower Bound: 0,
Q1: 611.5,
Mediana (Q2): 1207.0,
Q3: 1936.0,
Upper Bound: 3922.75


In [278]:
labels = ['ativo', 'moderadamente-ativo', 'inativo']

bins = [lower_bound, q1, q2, upper_bound]

df['contas']['class_inat'] = pd.cut(df['contas']['t_inat'], bins=bins, labels=labels, include_lowest=True)

In [279]:
df['contas']['class_inat'].value_counts()

inativo                486
ativo                  250
moderadamente-ativo    250
Name: class_inat, dtype: int64

### Propostas de Crédito

In [280]:
df['propostas_credito'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cod_proposta           2000 non-null   int64  
 1   cod_cliente            2000 non-null   int64  
 2   cod_colaborador        2000 non-null   int64  
 3   data_entrada_proposta  2000 non-null   object 
 4   taxa_juros_mensal      2000 non-null   float64
 5   valor_proposta         2000 non-null   float64
 6   valor_financiamento    2000 non-null   float64
 7   valor_entrada          2000 non-null   float64
 8   valor_prestacao        2000 non-null   float64
 9   quantidade_parcelas    2000 non-null   int64  
 10  carencia               2000 non-null   int64  
 11  status_proposta        2000 non-null   object 
dtypes: float64(5), int64(5), object(2)
memory usage: 187.6+ KB


#### Propostas de crédito por Colaborador

In [281]:
df['propostas_credito']['cod_colaborador'].value_counts()

18    34
99    33
73    33
27    31
37    30
      ..
35    11
9     11
8     10
57    10
50     8
Name: cod_colaborador, Length: 100, dtype: int64

#### Status das propostas de crédito

In [282]:
df['propostas_credito']['status_proposta'].value_counts()

Enviada                 527
Aprovada                514
Validação documentos    491
Em análise              468
Name: status_proposta, dtype: int64

### Transações

In [283]:
df['transacoes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71999 entries, 0 to 71998
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cod_transacao    71999 non-null  int64  
 1   num_conta        71999 non-null  int64  
 2   data_transacao   71999 non-null  object 
 3   nome_transacao   71999 non-null  object 
 4   valor_transacao  71999 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.7+ MB


#### Tratamento e criação da coluna de ano e mês para as análises

In [284]:
mes = []
ano = []

for i in range(len(df['transacoes']['data_transacao'])):
    mes.append(int(datetime.strptime(df['transacoes']['data_transacao'][i].split()[0], "%Y-%m-%d").strftime("%m")))
    ano.append(int(datetime.strptime(df['transacoes']['data_transacao'][i].split()[0], "%Y-%m-%d").strftime("%Y")))

df['transacoes']['ano'] = ano
df['transacoes']['mes'] = mes

# uma análise inicial dos dados permitiu identificar que os dados do ano 2023 estão incompletos, havendo informações referentes apenas ao primeiro trimestre, logo este ano será removido do conjunto de dados para as análises seguintes

df['transacoes'] = df['transacoes'].drop(df['transacoes'][df['transacoes']['ano'] == 2023].index) # exclusão dos dados referentes a 2023

#### Análise dos dados das agências

In [285]:
transac_por_conta = df['transacoes']['num_conta'].value_counts()

agencias = {}

for i in df['agencias']['cod_agencia'].unique():
    df_local = pd.DataFrame(df['contas'][df['contas']['cod_agencia'] == i]['cod_cliente'])
    df_local['num_trans'] = df_local['cod_cliente'].map(transac_por_conta)
    agencias[i] = df_local['num_trans'].sum(axis=0)

In [286]:
df['agencias']['num_trans'] = df['agencias']['cod_agencia'].map(agencias)

fig = px.bar(df['agencias'], x='nome', y='num_trans', color='tipo_agencia',
             labels={'nome': 'Agência', 'num_trans' : 'Volume de Transações'})

fig.update_layout(width=1200, height=600)

fig.show()

fig.write_image('plots_export/contas_agencia.png')
fig.write_html('plots_export_html/contas_agencia.html')


#### Tipo de transferência

In [287]:
tipo_transf = {}

anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    tipo_transf[a] = df['transacoes'][df['transacoes']['ano'] == a]['nome_transacao'].value_counts()

tipo_transf_df = pd.DataFrame(tipo_transf).reset_index()

tipo_transf_df = tipo_transf_df.rename(columns={'index': 'transacao'})

dados_long_1 = tipo_transf_df.melt(id_vars=["transacao"], var_name="ano", value_name="valor")

fig = px.bar(dados_long_1, x='transacao', y='valor', color='ano',
             labels={'valor': 'Quantidade', 'transacao': 'Tipo de Transação'},
             title='Distribuição dos Tipos de Transações')

fig.update_layout(width=1200, height=600)

fig.show()

fig.write_image('plots_export/tipos_trans.png')
fig.write_html('plots_export_html/tipos_trans.html')

## Entregas

### Entrega 3

Uma dimensão de datas bem construída viabiliza a elaboração de análises mais robustas com relação ao tempo. O BanVic quer começar seu Data Warehouse (DW) com uma dim_dates que atenda seus requisitos de análise. Utilizando uma dimensão de datas, responda às duas perguntas a seguir. Por fim, proponha outras duas análises utilizando a dim_dates que possam ter valor de negócio para o BanVic.

- Qual trimestre tem, em média, mais transações aprovadas e qual tem, também em média,  maior volume movimentado?

Dado que não há no banco de dados um indicativo de quais transações foram aprovadas ou rejeitadas, assume-se que todas as transações contidas na BD foram aprovadas.

In [288]:
num_transf = {}

meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    count = df['transacoes'][df['transacoes']['ano'] == a]['mes'].value_counts()

    for m in meses:
        if m not in list(count.keys()):
            count[m] = 0

    num_transf[a] = {1: count[1] + count[2] + count[3], 
                     2: count[4] + count[5] + count[6], 
                     3: count[7] + count[8] + count[9], 
                     4: count[10] + count[11] + count[12]}

num_transf_df = pd.DataFrame(num_transf)
num_transf_df = num_transf_df.reset_index()
num_transf_df = num_transf_df.rename(columns={'index': 'Trimestre'})

In [289]:
vol_transf = {}

anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    valor_prim_tri = 0
    valor_seg_tri = 0
    valor_ter_tri = 0
    valor_qua_tri = 0

    meses = sorted(df['transacoes'][df['transacoes']['ano'] == a]['mes'].unique(), key=int)

    df_new = df['transacoes'][df['transacoes']['ano'] == a]

    for m in meses:
        if m in [1, 2, 3]:
            valor_prim_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [4, 5, 6]:
            valor_seg_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [7, 8, 9]:
            valor_ter_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))
        elif m in [10, 11, 12]:
            valor_qua_tri += sum(abs(df_new[df_new['mes'] == m]['valor_transacao']))

    vol_transf[a] = {1: valor_prim_tri, 2: valor_seg_tri, 3: valor_ter_tri, 4: valor_qua_tri}

vol_transf_df = pd.DataFrame(vol_transf)
vol_transf_df = vol_transf_df.reset_index()
vol_transf_df = vol_transf_df.rename(columns={'index': 'Trimestre'})

In [290]:
media = {'Trimestre': [1, 2, 3, 4], 'Número de Transferências': [], 'Volume Transferido': []}

for i in range(4):
    media['Número de Transferências'].append(np.mean(list(num_transf_df[num_transf_df['Trimestre'] == i + 1].loc[:, 2010:2022].values)))
    media['Volume Transferido'].append(np.mean(list(vol_transf_df[vol_transf_df['Trimestre'] == i + 1].loc[:, 2010:2022].values)))

In [291]:
pd.DataFrame(media)

,Trimestre,Número de Transferências,Volume Transferido
0,1,788.769231,7.110626e+05
1,2,845.461538,6.644091e+05
2,3,937.692308,9.016052e+05
3,4,2923.461538,2.163221e+06


- Um analista sugeriu que, meses que contém R no seu nome (Ex: janeiRo), tem um padrão de quantidade de transações diferentes dos meses que não apresentam R no nome (Ex: junho). Apresente suas análises validando ou não essa afirmação.

In [292]:
contagem_2 = {}

meses = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun', 7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'}
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    contagem_2[a] = df['transacoes'][df['transacoes']['ano'] == a]['mes'].value_counts()
    contagem_2[a].columns = ['count']

data_3 = pd.DataFrame(contagem_2)
data_3 = data_3.reset_index()
data_3 = data_3.rename(columns={'index': 'mes'})
data_3 = data_3.fillna(0)

data_3['mes_nome'] = data_3['mes'].map(meses)

data_3 = data_3.drop(columns='mes')

cont = {'mes': list(data_3['mes_nome']), 'valores': list(data_3.sum(axis=1)), 'com_r_flag': [True, True, True, True, False, False, False, False, True, True, True, True]}

cont_df = pd.DataFrame(cont)

fig = px.bar(cont_df, x='mes', y='valores', color='com_r_flag', labels={'mes': 'Mês', 'valores': 'Quantidade de Transações', 'com_r_flag': 'Meses com R'})

fig.update_layout(barmode='stack', xaxis={'categoryorder': 'array', 'categoryarray': list(meses.values())}, 
                  title='Distribuição Mensal da Quantidade de Transações',
                  width=1200, height=600)

fig.show()

fig.write_image('plots_export/meses_com_r.png')
fig.write_html('plots_export_html/meses_com_r.html')

C:\Users\Adel\AppData\Local\Temp\ipykernel_10508\3318701122.py:19: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [293]:
dados_long_2 = data_3.melt(id_vars=["mes_nome"], var_name="Ano", value_name="Valor")

fig = px.bar(dados_long_2, x='mes_nome', y='Valor', labels={'mes_nome': 'Mês', 'Valor': 'Volume'}, 
             color='Ano', title="Transações por Mês por Ano")

fig.update_layout(barmode='stack', xaxis={'categoryorder': 'array', 'categoryarray': list(meses.values())}, 
                  title='Distribuição Mensal da Quantidade de Transações',
                  width=1200, height=600)

fig.show()

fig.write_image('plots_export/trans_mes_ano.png')
# fig.write_html('plots_export_html/meses_com_r.html')

#### Teste de Hipótese

Teste para verificar a variação do padrão nos meses que contém R no nome

In [294]:
from scipy.stats import ttest_ind, shapiro, mannwhitneyu

meses_com_r = data_3[data_3["mes_nome"].isin(["Jan", "Fev", "Mar", "Abr", "Set", "Out", "Nov", "Dez"])].drop(columns=["mes_nome"]).values.flatten()

meses_sem_r = data_3[data_3["mes_nome"].isin(["Mai", "Jun", "Jul", "Ago"])].drop(columns=["mes_nome"]).values.flatten()

shapiro_r = shapiro(meses_com_r)
shapiro_sem_r = shapiro(meses_sem_r)

print("Teste de Shapiro-Wilk para normalidade:")
print(f"Meses com 'R': p-valor = {shapiro_r.pvalue}")
print(f"Meses sem 'R': p-valor = {shapiro_sem_r.pvalue}")

if shapiro_r.pvalue > 0.05 and shapiro_sem_r.pvalue > 0.05:
    stat, p = ttest_ind(meses_com_r, meses_sem_r, equal_var=False)
    print("\nTeste t de Student:")
else:
    stat, p = mannwhitneyu(meses_com_r, meses_sem_r)
    print("\nTeste de Mann-Whitney:")

print(f"Estatística: {stat}, p-valor: {p}")

alpha = 0.05
if p < alpha:
    print("Há diferença significativa entre os grupos.")
else:
    print("Não há evidências suficientes para afirmar que há diferença.")


Teste de Shapiro-Wilk para normalidade:
Meses com 'R': p-valor = 6.204497294904409e-22
Meses sem 'R': p-valor = 0.0003104768693447113

Teste de Mann-Whitney:
Estatística: 2622.5, p-valor: 0.7607329165972996
Não há evidências suficientes para afirmar que há diferença.


### Entrega 4

- Um exemplo de dado externo é o Índice Nacional de Preços ao Consumidor Amplo. Considerando o IPCA no Brasil para o período apresentado na base de dados da BanVic, avalie se existe alguma relação do índice de preços com o volume de transações e/ou com o valor transacionado no período. Apresente suas análises e justifique sua opinião.

#### Carregamento dos dados do IPCA

In [295]:
ipca_data = pd.read_csv('dados/dados_auxiliar/dados_ipca.csv')
ipca_df = pd.DataFrame(ipca_data)

ipca_df = ipca_df[29:42] # considerando os anos de 2010 a 2021

ipca_df = ipca_df.set_index('Ano').T.reset_index()

ipca_df.columns.name = None

ipca_df = ipca_df.rename(columns={'index': 'Mes'})

#### Criação do dataframe dos dados do volume transferido

In [296]:
data_4 = df['transacoes'].loc[:, 'valor_transacao':'mes']

valor_trans = {'Mes' : ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']}

meses = [1,2,3,4,5,6,7,8,9,10,11,12]
anos = sorted(df['transacoes']['ano'].unique(), key=int)

for a in anos:
    valor_trans[a] = []
    for m in meses:
        valor_trans[a].append(sum(data_4[(data_4['ano'] == a) & (data_4['mes'] == m)]['valor_transacao']))

df_transf = pd.DataFrame(valor_trans) # dataframe de valores das transferencias (2010 a 2021)

df_transf = df_transf.loc[:, 'Mes': 2022]

#### Criação do novo dataframe com os dados do número de transações, volume transferido e IPCA anuais

In [297]:
def criar_df(inicio, fim):
    labels = ['IPCA', 'Valor Transferido', 'Número de Transferências']
    ipca_anual = ipca_df.loc[:, inicio:fim].sum(axis=0) # ipca anual
    valor_transf_ano = df_transf.loc[:, inicio:fim].sum(axis=0) # valores transferidos anualmente
    num_transf_anual = data_3.loc[:, inicio:fim].sum(axis=0) # número de transferências por ano

    df_un = pd.DataFrame([ipca_anual, valor_transf_ano, num_transf_anual])
    df_un.insert(0, 'ind', labels)

    df_un = df_un.set_index('ind').T.reset_index()

    df_un.columns.name = None

    df_un = df_un.rename(columns={'index': 'Ano'})

    return df_un

#### Criação do gráfico de correlação

In [298]:
def gerar_corr(df, ano):
    fig = px.imshow(df, text_auto=True,  # Exibe os valores dentro do heatmap
        color_continuous_scale="RdBu_r",  # Mapa de cores (azul-vermelho)
        labels=dict(color="Correlação")  # Nome da legenda de cores)
    )

    # Títulos e layout
    fig.update_layout(
        title=f"Matriz de Correlação entre IPCA e Indicadores (2010 a {ano})",
        xaxis_title="Variáveis",
        yaxis_title="Variáveis"
    )

    fig.show()
    # fig.write_image('plots_export/corr_ipca_2010_2022.png')

#### Checagem da correlação entre os três índices anuais através da correlação de Pearson

Fonte: https://www.blog.psicometriaonline.com.br/o-que-e-correlacao-de-pearson/

In [299]:
df_un_2022 = criar_df(2010, 2022)

matriz_corr_2022 = df_un_2022[['IPCA', 'Valor Transferido', 'Número de Transferências']].corr()

gerar_corr(matriz_corr_2022, 2022)

In [300]:
df_un_2021 = criar_df(2010, 2021)

matriz_corr_2021 = df_un_2021[['IPCA', 'Valor Transferido', 'Número de Transferências']].corr()

gerar_corr(matriz_corr_2021, 2021)

#### Normalização com Z-score

Fonte: https://www.investopedia.com/terms/z/zscore.asp

Motivo: Dados em escalas diferentes

2022

In [301]:
df_un_2022['Valor Transferido Normalizado'] = zscore(df_un_2022['Valor Transferido'])
df_un_2022['Número de Transferências Normalizado'] = zscore(df_un_2022['Número de Transferências'])
df_un_2022['IPCA Normalizado'] = zscore(df_un_2022['IPCA'])

df_un_edit = df_un_2022[['Ano', 'Valor Transferido Normalizado', 'Número de Transferências Normalizado', 'IPCA Normalizado']]

dados_long_6 = df_un_edit.melt(id_vars=['Ano'], var_name='Score', value_name='Valor')

fig = px.line(dados_long_6, x="Ano", y="Valor", color="Score",
              title="Valor Transferido, Número de Transferências e IPCA Normalizados por Z-score", 
              labels={'valor_transf_zscore': 'Valor Transferido Normalizado', 'num_transf_zscore': 'Número de Transferências Normalizado', 'ipca_zscore': 'IPCA Normalizado', 'Score': 'Legenda'}
              )

fig.update_layout(width=1200, height=600)

fig.show()

fig.write_image('plots_export/ipca_nt_vt_zscore_22.png')
fig.write_html('plots_export_html/ipca_nt_vt_zscore_22.html')

2021

In [302]:
df_un_2021['Valor Transferido Normalizado'] = zscore(df_un_2021['Valor Transferido'])
df_un_2021['Número de Transferências Normalizado'] = zscore(df_un_2021['Número de Transferências'])
df_un_2021['IPCA Normalizado'] = zscore(df_un_2021['IPCA'])

df_un_edit = df_un_2021[['Ano', 'Valor Transferido Normalizado', 'Número de Transferências Normalizado', 'IPCA Normalizado']]

dados_long_6 = df_un_edit.melt(id_vars=['Ano'], var_name='Score', value_name='Valor')

fig = px.line(dados_long_6, x="Ano", y="Valor", color="Score",
              title="Valor Transferido, Número de Transferências e IPCA Normalizados por Z-score", 
              labels={'valor_transf_zscore': 'Valor Transferido Normalizado', 'num_transf_zscore': 'Número de Transferências Normalizado', 'ipca_zscore': 'IPCA Normalizado', 'Score': 'Legenda'}
              )

fig.update_layout(width=1200, height=600)

fig.show()

fig.write_image('plots_export/ipca_nt_vt_zscore.png')